In [ ]:
import pandas as pd

In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/Research_SEM4/Dataset_AfterClean/train_df.csv")
dev_df = pd.read_csv("/content/drive/MyDrive/Research_SEM4/Dataset_AfterClean/dev_df.csv")

In [ ]:
train_df.head()

,Unnamed: 0,original_text,extractive_summary
0,42855,jakarta jennifer arnelita datang ke sebuah...,ia menjadi sutradara dalam penggarapan video k...
1,136345,jakarta abu bakar ba asyir tidak kaget men...,jakarta abu bakar ba asyir tidak kaget men...
2,88528,jakarta menteri negara pembangunan nasiona...,alasannya kwik mengaku sudah mengantongi izin...
3,85788,garut ratusan warga dari berbagai ormas is...,garut ratusan warga dari berbagai ormas is...
4,42360,jakarta mantan ketua kssk sri mulyani indr...,jakarta mantan ketua kssk sri mulyani indr...


In [ ]:
dev_df.head()

,Unnamed: 0,original_text,extractive_summary
0,7434,solok untuk menghindari pembengkakan angga...,menurut gamawan sulit bagi kabupaten solok me...
1,10428,jakarta kenaikan harga sembilan bahan poko...,harga sembako mulai melonjak sejak distribusi ...
2,8617,medan anggota intel pangkalan utama tni an...,medan anggota intel pangkalan utama tni an...
3,360,jakarta panglima tni laksamana widodo as m...,jakarta panglima tni laksamana widodo as m...
4,6814,jakarta pemerintah tengah menyiapkan keput...,jakarta pemerintah tengah menyiapkan keput...


In [ ]:
train_df = train_df.drop(['Unnamed: 0'], axis=1)
dev_df = dev_df.drop(['Unnamed: 0'], axis=1)

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23220 entries, 0 to 23219
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   original_text       23220 non-null  object
 1   extractive_summary  23220 non-null  object
dtypes: object(2)
memory usage: 362.9+ KB


In [ ]:
dev_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3004 entries, 0 to 3003
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   original_text       3004 non-null   object
 1   extractive_summary  3004 non-null   object
dtypes: object(2)
memory usage: 47.1+ KB


# Finetuning model


In [ ]:
!pip install datasets transformers accelerate torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
import torch
from transformers import BertTokenizer, BertForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from torch.utils.data import Dataset
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Device Name: {torch.cuda.get_device_name(0)}")

PyTorch Version: 2.6.0+cu124
CUDA Available: True
Device Name: Tesla T4


In [ ]:
device = 'cuda'

In [ ]:
model_name = "bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForTokenClassification.from_pretrained(
    model_name,
    num_labels=2  # Binary classification: part of summary (1) or not (0)
).to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_dataset = Dataset.from_pandas(train_df)
dev_dataset = Dataset.from_pandas(dev_df)

In [ ]:
def preprocess_function_extractive(examples):
    """
    Tokenizes inputs and creates token-level labels based on whether tokens
    appear in the extractive summary with improved alignment using offset mapping.
    """
    # Ensure extractive_summary is a string
    extractive_summary = [
        " ".join(summary) if isinstance(summary, list) else summary
        for summary in examples["extractive_summary"]
    ]

    # Tokenize the input text with return_offsets_mapping=True to get character-level alignments
    model_inputs = tokenizer(
        examples["original_text"],
        max_length=512,
        truncation=True,
        padding="max_length"
    )

    # Create empty label arrays (0 = not in summary, 1 = in summary)
    labels = []

    for i, (text, summary) in enumerate(zip(examples["original_text"], extractive_summary)):
        # Get input IDs for this example
        input_ids = model_inputs["input_ids"][i]

        # Convert input_ids to tokens
        tokens = tokenizer.convert_ids_to_tokens(input_ids)

        # Get summary tokens
        summary_tokens = tokenizer.tokenize(summary)

        # Initialize labels with special token handling
        example_labels = []

        for j, token in enumerate(tokens):
            # Mark special tokens with -100 to be ignored in loss calculation
            if token in [tokenizer.cls_token, tokenizer.sep_token, tokenizer.pad_token]:
                example_labels.append(-100)
            else:
                # Check if this token appears in the summary
                is_in_summary = 1 if token in summary_tokens else 0
                example_labels.append(is_in_summary)

        labels.append(example_labels)

    # Add the labels to the model inputs
    model_inputs["labels"] = labels
    return model_inputs

In [ ]:
tokenized_train_dataset = train_dataset.map(preprocess_function_extractive, batched=True)
tokenized_dev_dataset = dev_dataset.map(preprocess_function_extractive, batched=True)

Map:   0%|          | 0/23220 [00:00<?, ? examples/s]

Map:   0%|          | 0/3004 [00:00<?, ? examples/s]

In [ ]:
print(train_dataset[0])
print(tokenized_train_dataset[0])

{'original_text': '   jakarta  jennifer arnelita datang ke sebuah studio pembuatan video klip  namun  kali ini jennifer tidak menjadi model  ia menjadi sutradara dalam penggarapan video klip untuk grup musik anyar  debra band  lantas  apakah jennifer telah beralih profesi dari dunia artis ke musik   cuma side job aja kok  ini pertama kalinya gue jadi sutradara buat video klip  rasanya seneng banget bisa dapetin pengalaman baru  pokoknya enak deh bisa nyuruh nyuruh  berasa punya anak buah gue  kata jennifer sembari tertawa seperti ditayangkan status selebritis  rabu  14 7   jennifer pun menjelaskan tentang penggarapan video klip ini   kenapa ngambilnya studio  karena kita syutingnya dua hari  hari pertama untuk anak band nya sendiri  hari kedua baru story line nya untuk cerita di video klip ini  ujar jen  video klip itu menceritakan kisah seorang laki laki yang merasa ditindas oleh pasangannya  begitu jalinan cinta di antara keduanya putus  sang adam menemukan dunianya kembali   yus  ',

In [ ]:
data_collator = DataCollatorForTokenClassification(
    tokenizer
)

In [ ]:
def compute_metrics(eval_pred):
    """
    Compute metrics for token classification with improved token-to-text precision.
    """
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=2)

    # Only keep predictions for non-masked tokens
    true_predictions = []
    true_labels = []

    for prediction, label in zip(predictions, labels):
        for p, l in zip(prediction, label):
            if l != -100:  # Skip masked tokens (-100)
                true_predictions.append(p)
                true_labels.append(l)

    # Calculate token-level metrics
    precision, recall, f1, _ = precision_recall_fscore_support(
        true_labels, true_predictions, average='binary'
    )
    acc = accuracy_score(true_labels, true_predictions)

    # Add token-to-text precision metric
    # This would normally require the original text and token spans to calculate
    # For now, we'll just use the binary metrics as a placeholder

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

# Define training arguments - fixed for compatibility
training_args = TrainingArguments(
    output_dir="./bert_extractive_summarization",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    learning_rate=2e-5,
    gradient_accumulation_steps=4,
    eval_strategy="steps",
    eval_steps=500,
    save_total_limit=3,
    save_strategy="steps",  # or save_steps for older versions
    save_steps=500,
    logging_dir='./logs',
    logging_steps=100,
    report_to="none",
)

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_dev_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Train model
trainer.train()

# Save model
model.save_pretrained("./bert_extractive_summarization/final_model")
tokenizer.save_pretrained("./bert_extractive_summarization/final_model")
print("Model and tokenizer saved successfully!")

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
500,0.482300,0.501166,0.763462,0.696691,0.639397,0.666816
1000,0.467700,0.499332,0.763792,0.694104,0.647105,0.669781
1500,0.452900,0.519671,0.758750,0.656377,0.730982,0.691673
2000,0.444800,0.534409,0.757172,0.651487,0.739786,0.692835
2500,0.439600,0.505268,0.764055,0.668815,0.718339,0.692693
3000,0.419600,0.507809,0.762399,0.670118,0.705417,0.687315
3500,0.421700,0.517917,0.758043,0.656952,0.724945,0.689275
4000,0.422800,0.513863,0.760684,0.665790,0.709854,0.687116


Model and tokenizer saved successfully!


In [ ]:
import os
output_dir = '/content/drive/MyDrive/Research_SEM4/Hasil_FineTuning/mBert:Extractive'

# Create the directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the trained model and tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Model saved to {output_dir}")

Model saved to /content/drive/MyDrive/Research_SEM4/Hasil_FineTuning/mBert:Extractive
